In [ ]:
import pandas as pd
import numpy as np
import re
import requests
import xmltodict
import time
import pickle
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from lxml import html

In [ ]:
#pd.set_option('display.max_colwidth', -1)

In [ ]:
DETAILED_LINK_URL = "https://data.rechtspraak.nl/uitspraken/content?id={}"


In [ ]:
titles_df = pd.read_csv("structured_cases.CSV")

In [ ]:
ECLI_list = titles_df['ECLI'].tolist()

In [ ]:
def retreive_title(ECLI_list):
    
    detailed_link_list = [ecli for ecli in ECLI_list]
      
    text_df = pd.DataFrame({
        "ECLI": [],
        "full_text": [],
        "titles": []

    })
    
    for detailed_link in detailed_link_list:
        
        ecli_temp = get_ecli_from_detailed_link(detailed_link)

        
        detailed_sentence = requests.get(
            DETAILED_LINK_URL.format(ecli_temp))
    
        text = get_html_from_detailed_link(detailed_sentence)
        
        try:
            uitspraak = [
                x for x in text.getchildren()
                if "uitspraak" in str(x).lower()
            ][0]
            
            full_text = get_full_text(text)
            titles = get_titles_from_html(uitspraak)
            
        
        except IndexError as i:
            full_text = ["No Full Text"]
            
        df_temp = pd.DataFrame({
            "ECLI": [ecli_temp],
            "full_text": [full_text],
            "titles": [titles]
        })
    
        text_df = text_df.append(df_temp, ignore_index=True)
        
    return text_df

    
    
def get_html_from_detailed_link(called_api_page):
    try:
        detailed_page_element_tree = html.fromstring(called_api_page.content)
    except Exception as e:
        print("{error} happend".format(error=e))
    return detailed_page_element_tree


def get_full_text(detailed_sentence):
    uitspraak = detailed_sentence.getchildren()[2]
    return uitspraak.text_content().lower().replace("\r\n", " ").strip()

def get_ecli_from_detailed_link(detailed_link):
    try:
        return detailed_link.split("=")[-1]
    except:
        return None
    
def get_titles_from_html(uitspraak):
    try:
        sections = [
            x for x in uitspraak.getchildren() if "section" in str(x).lower()
        ]
        titles = [
            ''.join([
                i for i in sec.findall("title")[0].text_content().replace(
                    "\r\n      ", "").strip() if not i.isdigit()
            ]) for sec in sections if sec.findall("title")
        ]
        return [
            t.lower().strip()
            for t in titles
        ]
    except Exception as e:
        return ["No Uitspraak"]



In [ ]:
retreive_title(ECLI_list)

In [ ]:
#retrive the unique titles 

titles = ['procesverloop', 'overwegingen', 'beslissing']

for index, row in titles_df.iterrows():
    print(row.titles)
    unique_titles = list(set().union(row.titles, titles))
    titles = unique_titles